In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

# SPDX-License-Identifier: Apache-2.0

#

# Licensed under the Apache License, Version 2.0 (the "License");

# you may not use this file except in compliance with the License.

# You may obtain a copy of the License at

#

# http://www.apache.org/licenses/LICENSE-2.0

#

# Unless required by applicable law or agreed to in writing, software

# distributed under the License is distributed on an "AS IS" BASIS,

# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

# See the License for the specific language governing permissions and

# limitations under the License.

In [1]:
### This notebook reads the dgl graph and does preprocessing according to data_graph.py for the model to read
import os, sys
ROOT_DIR = '/raid/andlai/2024_ICCAD_Contest_Gate_Sizing_Benchmark'
sys.path.append(ROOT_DIR)
import time

import pickle as pk

import numpy as np
import pandas as pd

import dgl
import networkx as nx
# import graph_tool as gt
# from graph_tool.all import *

import torch

import matplotlib.pyplot as plt

/raid/yba/miniconda3/envs/andlai_dgl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## load all datasets in design_names = ['NV_NVDLA_partition_m', 'NV_NVDLA_partition_p', 'ariane136', 'mempool_tile_wrap']
design_names = ['NV_NVDLA_partition_m', 'NV_NVDLA_partition_p', 'ariane136', 'mempool_tile_wrap'] + ['ac97_top', 'aes', 'aes_cipher_top', 'des', 'des3', 'i2c_master_top', 'mc_top', 'pci_bridge32', 'tv80s']
dataset_dir = '0820_v1'
testing_designs = ['NV_NVDLA_partition_m']

# read all the graph for all the designs
gs = {}
for design_name in design_names:
    design_dir = f'{ROOT_DIR}/datasets/{dataset_dir}/{design_name}'
    gs[design_name] = dgl.load_graphs(f'{design_dir}/graph.dgl')[0]

# identical function taken from data_graph.py
def gen_topo(g_hetero):
    torch.cuda.synchronize()
    time_s = time.time()
    na, nb = g_hetero.edges(etype='net_out', form='uv')
    ca, cb = g_hetero.edges(etype='cell_out', form='uv')
    g = dgl.graph((torch.cat([na, ca]).cpu(), torch.cat([nb, cb]).cpu()))
    topo = dgl.topological_nodes_generator(g)

    ### inspect the topography!
    g.ndata['fanout'] = g_hetero.ndata['nf'][:, 1].cpu()
    for li, nodes in enumerate(topo):
        # print(f'level {li}, # nodes = {len(nodes)}')
        # print(g.ndata['fanout'][nodes.numpy()])
        assert (li % 2 == 0 and (g.ndata['fanout'][nodes] == 0).sum() == 0) or (li % 2 == 1 and (g.ndata['fanout'][nodes] == 1).sum() == 0)

    assert len(topo) % 2 == 0

    ret = [t.cuda() for t in topo]
    torch.cuda.synchronize()
    time_e = time.time()
    return ret, time_e - time_s

# data preprocessing
new_gs = {}
for design_name, des_gs in gs.items():
    new_gs[design_name] = []
    for gi, g in enumerate(des_gs):
        g.nodes['node'].data['nf'] = g.nodes['node'].data['nf'].type(torch.float32)
        g.edges['cell_out'].data['ef'] = g.edges['cell_out'].data['ef'].type(torch.float32)
        g.edges['net_out'].data['ef'] = g.edges['net_out'].data['ef'].type(torch.float32)
        g.edges['net_in'].data['ef'] = g.edges['net_in'].data['ef'].type(torch.float32)

        g.ndata['n_slack'][g.ndata['n_slack'] > 1e10] = torch.nan
        g.ndata['n_risearr'][g.ndata['n_risearr'] < -1e10] = torch.nan
        g.ndata['n_fallarr'][g.ndata['n_fallarr'] < -1e10] = torch.nan
        g.ndata['n_tsrf'] = torch.stack([g.ndata['n_tran'], g.ndata['n_slack'], g.ndata['n_risearr'], g.ndata['n_fallarr']], axis=1).type(torch.float32)

        # print(f'{design_name}, {gi+1}/{len(des_gs)}')
        topo, topo_time = gen_topo(g)

        ts = {'input_nodes': (g.ndata['nf'][:, 1] < 0.5).nonzero().flatten().type(torch.int32),
            'output_nodes': (g.ndata['nf'][:, 1] > 0.5).nonzero().flatten().type(torch.int32),
            'output_nodes_nonpi': torch.logical_and(g.ndata['nf'][:, 1] > 0.5, g.ndata['nf'][:, 0] < 0.5).nonzero().flatten().type(torch.int32),
            'pi_nodes': torch.logical_and(g.ndata['nf'][:, 1] > 0.5, g.ndata['nf'][:, 0] > 0.5).nonzero().flatten().type(torch.int32),
            # 'po_nodes': torch.logical_and(g.ndata['nf'][:, 1] < 0.5, g.ndata['nf'][:, 0] > 0.5).nonzero().flatten().type(torch.int32),
            # 'endpoints': (g.ndata['n_is_end'] > 0.5).nonzero().flatten().type(torch.long),
            'topo': [t.cpu() for t in topo]}
            # 'topo_time': topo_time}

        for key in ts.keys():
            if type(ts[key]) == torch.Tensor:
                ts[key] = ts[key].cpu().numpy()
            else:
                ts[key] = [item.cpu().numpy() for item in ts[key]]

        new_gs[design_name].append([g, ts])
        
# get means and stds
nf, n_tsrf, ef_pnop, ef_pcop = [], [], [], []
for design_name, des_gs in new_gs.items():
    if design_name in testing_designs:
        print(f'Skipped {design_name} for mean,std calculation')
        continue
    for gi, g_n_ts in enumerate(des_gs):
        g, ts = g_n_ts
        # get means and stds
        nf.append(g.ndata['nf'])
        # n_tsrf.append(g.ndata['n_tsrf'])
        ef_pnop.append(g.edata['ef'][('node', 'net_out', 'node')])
        ef_pcop.append(g.edata['ef'][('node', 'cell_out', 'node')])
nf = torch.cat(nf, axis=0)
# n_tsrf = torch.cat(n_tsrf, axis=0)
ef_pnop = torch.cat(ef_pnop, axis=0)
ef_pcop = torch.cat(ef_pcop, axis=0)

nf_mean = torch.nanmean(nf, dim=0)
nf_std = (torch.nanmean(nf ** 2, dim=0) - nf_mean ** 2) ** 0.5
# n_tsrf_mean = torch.nanmean(n_tsrf, dim=0)
# n_tsrf_std = (torch.nanmean(n_tsrf ** 2, dim=0) - n_tsrf_mean ** 2) ** 0.5
ef_pnop_mean = torch.nanmean(ef_pnop, dim=0)
ef_pnop_std = (torch.nanmean(ef_pnop ** 2, dim=0) - ef_pnop_mean ** 2) ** 0.5
ef_pcop_mean = torch.nanmean(ef_pcop, dim=0)
ef_pcop_std = (torch.nanmean(ef_pcop ** 2, dim=0) - ef_pcop_mean ** 2) ** 0.5
ef_pcop_std[torch.isnan(ef_pcop_std)] = 0
drop_ef_pcop_cols = ef_pcop_std == 0
print('There are', (ef_pcop_std == 0).sum(), 'all-the-same pin-cell-pin attributes')
ef_pcop_std[ef_pcop_std == 0] = ef_pcop_mean[ef_pcop_std == 0]

data = {}
for design_name, des_gs in new_gs.items():
    for gi, g_n_ts in enumerate(des_gs):
        g, ts = g_n_ts
        topo = ts['topo']
        ## normalize
        g.ndata['nf'][:, :2] = g.ndata['nf'][:, :2] / nf_std[:2].unsqueeze(0)
        g.ndata['nf'][:, 2:] = (g.ndata['nf'][:, 2:] - nf_mean[2:].unsqueeze(0)) / nf_std[2:].unsqueeze(0)

        # self normalize for the answers
        n_tsrf_mean = torch.nanmean(g.ndata['n_tsrf'], dim=0)
        n_tsrf_std = (torch.nanmean(g.ndata['n_tsrf'] ** 2, dim=0) - n_tsrf_mean ** 2) ** 0.5
        g.ndata['n_tsrf'] /= n_tsrf_std.unsqueeze(0)

        g.edata['ef'] = {
            ('node', 'net_out', 'node') : g.edata['ef'][('node', 'net_out', 'node')] / ef_pnop_std.unsqueeze(0),
            ('node', 'net_in', 'node')  : g.edata['ef'][('node', 'net_in', 'node')] / ef_pnop_std.unsqueeze(0),
            ('node', 'cell_out', 'node'): g.edata['ef'][('node', 'cell_out', 'node')] / ef_pcop_std.unsqueeze(0)
        }

        # train mask must not contain nans
        assert (torch.isnan(g.ndata['nf'].any(dim=-1)) & g.ndata['train_mask']).sum() == 0
        assert (torch.isnan(g.ndata['n_tsrf'].any(dim=-1)) & g.ndata['train_mask']).sum() == 0

        # set nans to zero
        g.ndata['nf'][torch.isnan(g.ndata['nf'])] = 0
        g.ndata['n_tsrf'][torch.isnan(g.ndata['n_tsrf'])] = 0
        
        data[(design_name, gi)] = g, ts

        # some assertions
        assert ts['input_nodes'].max() < len(g.ndata['nf']), f"{ts['input_nodes'].max()}, {g.ndata['nf'].shape}"
        assert ts['output_nodes'].max() < len(g.ndata['nf']), f"{ts['output_nodes'].max()}, {g.ndata['nf'].shape}"
        assert ts['output_nodes_nonpi'].max() < len(g.ndata['nf']), f"{ts['output_nodes_nonpi'].max()}, {g.ndata['nf'].shape}"
        assert ts['pi_nodes'].max() < len(g.ndata['nf']), f"{ts['pi_nodes'].max()}, {g.ndata['nf'].shape}"

        # just for report
        print(gi, design_name, len(g.ndata['nf']), g.ndata['train_mask'].sum().item(), len(g.edata['ef'][('node', 'cell_out', 'node')]), len(g.edata['ef'][('node', 'net_out', 'node')]), len(topo))

data_train = {k: t for k, t in data.items() if k[0] not in testing_designs}
data_test = {k: t for k, t in data.items() if k[0] in testing_designs}

# normalize LUT props
with open(f'{ROOT_DIR}/datasets/{dataset_dir}/LUT_prop.pk', 'rb') as pkf:
    LUT_prop = pk.load(pkf)
for key in LUT_prop.keys():
    LUT_prop[key] /= ef_pcop_std.numpy()
with open(f'{ROOT_DIR}/datasets/{dataset_dir}/LUT_prop_normed.pk', 'wb') as pkf:
    pk.dump(LUT_prop, pkf)

# normalize rise fall cap props
rf_cap = pd.read_csv(f'{ROOT_DIR}/datasets/{dataset_dir}/rise_fall_caps.csv', index_col=0)
rf_cap = (rf_cap - nf_mean[-2:].unsqueeze(0).numpy()) / nf_std[-2:].unsqueeze(0).numpy()
rf_cap.to_csv(f'{ROOT_DIR}/datasets/{dataset_dir}/rise_fall_caps_normed.csv')

Skipped NV_NVDLA_partition_m for mean,std calculation
There are tensor(32) all-the-same pin-cell-pin attributes
0 NV_NVDLA_partition_m 83671 79658 54280 55966 86
1 NV_NVDLA_partition_m 458 312 264 298 16
0 NV_NVDLA_partition_p 273679 232889 171386 193361 82
0 ariane136 499589 443071 329085 352154 190
0 mempool_tile_wrap 657377 612411 447458 468169 184
0 ac97_top 24427 18142 15143 17366 24
0 aes 16891 15552 11245 11888 60
0 aes_cipher_top 37899 36585 26415 27288 44
0 des 9240 8925 6167 6357 46
0 des3 8394 8138 5909 6038 56
0 i2c_master_top 1774 1531 1083 1208 40
0 mc_top 20083 17222 12824 13800 58
0 pci_bridge32 38302 30420 23132 27748 50
0 tv80s 12539 11823 8776 9092 72


In [3]:
print('[node data] ( = dstdata)')
for nkey, ndat in g.ndata.items():
    assert type(ndat) == torch.Tensor, 'Type must be torch.Tensor'
    print(f'{nkey:22s} {ndat.shape}')
    if nkey == 'nf':
        nf = ndat
        for fkey, frange in [('is_prim IO', [0,1]), ('fanout(1) or in(0)', [1,2]), ('dis to tlrb', [2,6]), ('RF cap', [6,8])]:
            print(f'  {fkey:20s} {ndat[:, frange[0]:frange[1]].shape}')
print()

print('[edge data]')
for ekey, edat in g.edata.items():
    assert type(edat) == dict, 'Type must be dict'
    print(f'{ekey}:')
    for edat_key, edat_dat in edat.items():
        print(f'  {f"{edat_key}":30s} {edat_dat.shape}')

[node data] ( = dstdata)
nf                     torch.Size([12539, 8])
  is_prim IO           torch.Size([12539, 1])
  fanout(1) or in(0)   torch.Size([12539, 1])
  dis to tlrb          torch.Size([12539, 4])
  RF cap               torch.Size([12539, 2])
n_tran                 torch.Size([12539])
n_risearr              torch.Size([12539])
n_slack                torch.Size([12539])
n_fallarr              torch.Size([12539])
n_is_end               torch.Size([12539])
train_mask             torch.Size([12539])
cell_id                torch.Size([12539])
_ID                    torch.Size([12539])
n_tsrf                 torch.Size([12539, 4])

[edge data]
ef:
  ('node', 'cell_out', 'node')   torch.Size([8776, 256])
  ('node', 'net_in', 'node')     torch.Size([9092, 5])
  ('node', 'net_out', 'node')    torch.Size([9092, 5])
cell_id:
  ('node', 'cell_out', 'node')   torch.Size([8776])


In [4]:
with open(f'{ROOT_DIR}/project/TimingGCN_pl/data/{dataset_dir}/data_train.pk', 'wb') as pkf:
    pk.dump(data_train, pkf)
with open(f'{ROOT_DIR}/project/TimingGCN_pl/data/{dataset_dir}/data_test.pk',  'wb') as pkf:
    pk.dump(data_test, pkf)

{('NV_NVDLA_partition_m',
  0): (Graph(num_nodes={'node': 83671},
        num_edges={('node', 'cell_out', 'node'): 54280, ('node', 'net_in', 'node'): 55966, ('node', 'net_out', 'node'): 55966},
        metagraph=[('node', 'node', 'cell_out'), ('node', 'node', 'net_in'), ('node', 'node', 'net_out')]), {'input_nodes': array([    0,     2,     3, ..., 83363, 83364, 83365], dtype=int32),
   'output_nodes': array([    1,     5,    10, ..., 83668, 83669, 83670], dtype=int32),
   'output_nodes_nonpi': array([    1,     5,    10, ..., 83358, 83362, 83366], dtype=int32),
   'pi_nodes': array([30327, 37917, 37919, 37921, 37923, 37925, 37927, 37929, 37931,
          37933, 37935, 37937, 37939, 37941, 37943, 37945, 37947, 37949,
          37951, 37953, 37993, 37995, 37997, 37999, 38001, 38003, 38005,
          38007, 38009, 38011, 38013, 38015, 38017, 38019, 38021, 38025,
          38027, 38029, 38069, 38071, 38073, 38075, 38077, 38079, 38081,
          38083, 38085, 38087, 38089, 38091, 38093, 38

In [14]:
# distribution of the variables in data train
for des, (g, ts) in data_test.items():
    print([len(topo )for topo in ts['topo']])

[1865, 5551, 1048, 4569, 2102, 5736, 2900, 5245, 1997, 3730, 1823, 3338, 1656, 3056, 1436, 2532, 1281, 2126, 1109, 1781, 868, 1545, 801, 1430, 754, 1381, 716, 1299, 700, 1312, 692, 1282, 674, 1104, 520, 778, 412, 670, 420, 690, 413, 737, 426, 751, 358, 682, 330, 549, 290, 529, 256, 514, 261, 480, 257, 428, 202, 350, 177, 292, 156, 266, 142, 225, 121, 197, 100, 170, 104, 164, 95, 144, 67, 98, 48, 75, 46, 67, 40, 49, 27, 29, 13, 13, 2, 2]
[65, 123, 18, 53, 30, 47, 15, 24, 10, 15, 7, 17, 7, 11, 8, 8]
